<a href="https://colab.research.google.com/github/hackershannu/DRUG_INTERACTION/blob/main/Another_copy_of_DRUG_INTERACTION_DETCTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess
import sys

def install_packages():
    packages = [
        'gradio',
        'transformers',
        'torch',
        'pandas',
        'numpy',
        'nltk',
        'requests'
    ]

    for package in packages:
        try:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        except subprocess.CalledProcessError:
            print(f"Failed to install {package}")

# Install packages automatically
install_packages()

Installing gradio...
Installing transformers...
Installing torch...
Installing pandas...
Installing numpy...
Installing nltk...
Installing requests...


In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import json
import re
import nltk
import requests
from typing import List, Dict, Any, Tuple
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Drug Interaction Detection System - Simple Version
# Uses Hugging Face Transformers (no API key required)


# Hugging Face Transformers for local AI
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Download NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
except:
    pass

class DrugDatabase:
    """Enhanced drug database with interaction data"""

    def __init__(self):
        self.drugs_data = {
            'aspirin': {
                'generic_name': 'acetylsalicylic acid',
                'category': 'NSAID',
                'interactions': ['warfarin', 'heparin', 'methotrexate', 'alcohol'],
                'contraindications': ['bleeding disorders', 'peptic ulcer', 'asthma'],
                'age_restrictions': {'min_age': 12, 'pregnancy_category': 'D'},
                'dosage_forms': ['tablet', 'capsule', 'chewable'],
                'common_dosages': {'adult': '325-650mg every 4-6 hours', 'pediatric': '10-15mg/kg every 4 hours'}
            },
            'warfarin': {
                'generic_name': 'warfarin sodium',
                'category': 'anticoagulant',
                'interactions': ['aspirin', 'ibuprofen', 'simvastatin', 'amiodarone', 'alcohol'],
                'contraindications': ['active bleeding', 'pregnancy', 'severe liver disease'],
                'age_restrictions': {'min_age': 18, 'pregnancy_category': 'X'},
                'dosage_forms': ['tablet'],
                'common_dosages': {'adult': '2-10mg daily (individualized)', 'elderly': '1-5mg daily'}
            },
            'ibuprofen': {
                'generic_name': 'ibuprofen',
                'category': 'NSAID',
                'interactions': ['warfarin', 'lisinopril', 'methotrexate', 'aspirin'],
                'contraindications': ['kidney disease', 'heart failure', 'bleeding disorders'],
                'age_restrictions': {'min_age': 6, 'pregnancy_category': 'C'},
                'dosage_forms': ['tablet', 'liquid', 'injection', 'gel'],
                'common_dosages': {'adult': '200-800mg every 6-8 hours', 'pediatric': '5-10mg/kg every 6-8 hours'}
            },
            'lisinopril': {
                'generic_name': 'lisinopril',
                'category': 'ACE inhibitor',
                'interactions': ['ibuprofen', 'potassium supplements', 'lithium', 'alcohol'],
                'contraindications': ['angioedema history', 'pregnancy', 'bilateral renal artery stenosis'],
                'age_restrictions': {'min_age': 6, 'pregnancy_category': 'D'},
                'dosage_forms': ['tablet'],
                'common_dosages': {'adult': '5-40mg daily', 'pediatric': '0.1-0.6mg/kg daily'}
            },
            'metformin': {
                'generic_name': 'metformin hydrochloride',
                'category': 'antidiabetic',
                'interactions': ['contrast dye', 'alcohol', 'cimetidine'],
                'contraindications': ['kidney disease', 'metabolic acidosis', 'severe heart failure'],
                'age_restrictions': {'min_age': 10, 'pregnancy_category': 'B'},
                'dosage_forms': ['tablet', 'extended-release', 'liquid'],
                'common_dosages': {'adult': '500-2000mg daily', 'pediatric': '500-2000mg daily'}
            },
            'simvastatin': {
                'generic_name': 'simvastatin',
                'category': 'statin',
                'interactions': ['warfarin', 'digoxin', 'amiodarone', 'grapefruit juice'],
                'contraindications': ['liver disease', 'pregnancy', 'active muscle disease'],
                'age_restrictions': {'min_age': 18, 'pregnancy_category': 'X'},
                'dosage_forms': ['tablet'],
                'common_dosages': {'adult': '5-80mg daily (evening)', 'elderly': '5-40mg daily'}
            },
            'acetaminophen': {
                'generic_name': 'acetaminophen',
                'category': 'analgesic/antipyretic',
                'interactions': ['warfarin', 'alcohol', 'phenytoin'],
                'contraindications': ['severe liver disease'],
                'age_restrictions': {'min_age': 0, 'pregnancy_category': 'B'},
                'dosage_forms': ['tablet', 'liquid', 'suppository', 'injection'],
                'common_dosages': {'adult': '325-1000mg every 4-6 hours', 'pediatric': '10-15mg/kg every 4-6 hours'}
            }
        }

        # Alternative medications
        self.alternatives = {
            'aspirin': ['acetaminophen', 'celecoxib', 'naproxen'],
            'warfarin': ['rivaroxaban', 'apixaban', 'dabigatran'],
            'ibuprofen': ['naproxen', 'celecoxib', 'acetaminophen'],
            'lisinopril': ['losartan', 'enalapril', 'ramipril'],
            'simvastatin': ['atorvastatin', 'rosuvastatin', 'pravastatin'],
            'metformin': ['gliclazide', 'glipizide', 'pioglitazone']
        }

        # Interaction severity levels
        self.interaction_severity = {
            ('aspirin', 'warfarin'): 'high',
            ('ibuprofen', 'warfarin'): 'high',
            ('aspirin', 'ibuprofen'): 'moderate',
            ('lisinopril', 'ibuprofen'): 'moderate'
        }

    def get_drug_info(self, drug_name: str) -> Dict:
        """Get drug information from database"""
        drug_name = drug_name.lower().strip()
        return self.drugs_data.get(drug_name, {})

    def get_alternatives(self, drug_name: str) -> List[str]:
        """Get alternative medications"""
        drug_name = drug_name.lower().strip()
        return self.alternatives.get(drug_name, [])

    def check_interactions(self, drugs: List[str]) -> List[Dict]:
        """Check for drug interactions with severity levels"""
        interactions = []
        drugs = [drug.lower().strip() for drug in drugs]

        for i, drug1 in enumerate(drugs):
            drug1_info = self.get_drug_info(drug1)
            if not drug1_info:
                continue

            for j, drug2 in enumerate(drugs[i+1:], i+1):
                if drug2 in drug1_info.get('interactions', []):
                    # Get severity level
                    severity = self.interaction_severity.get((drug1, drug2)) or \
                              self.interaction_severity.get((drug2, drug1)) or 'moderate'

                    interactions.append({
                        'drug1': drug1,
                        'drug2': drug2,
                        'severity': severity,
                        'description': self._get_interaction_description(drug1, drug2, severity)
                    })

        return interactions

    def _get_interaction_description(self, drug1: str, drug2: str, severity: str) -> str:
        """Get detailed interaction description"""
        descriptions = {
            ('aspirin', 'warfarin'): 'Increased bleeding risk due to additive anticoagulant effects',
            ('ibuprofen', 'warfarin'): 'NSAIDs can increase warfarin effects and bleeding risk',
            ('aspirin', 'ibuprofen'): 'Concurrent use may reduce cardioprotective effects of aspirin',
            ('lisinopril', 'ibuprofen'): 'NSAIDs may reduce ACE inhibitor effectiveness and increase kidney risk'
        }

        key = (drug1, drug2) if (drug1, drug2) in descriptions else (drug2, drug1)
        return descriptions.get(key, f'Known interaction between {drug1} and {drug2}')

class LocalAIModel:
    """Local AI model using Hugging Face transformers - no API key needed"""

    def __init__(self):
        print("Loading AI model... This may take a few minutes the first time.")
        try:
            # Use a smaller, faster model that works well for medical text
            model_name = "microsoft/DialoGPT-medium"
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForCausalLM.from_pretrained(model_name)

            # Add padding token if not present
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            print("✅ AI model loaded successfully!")
        except Exception as e:
            print(f"⚠️ Error loading AI model: {e}")
            print("Using rule-based responses as fallback.")
            self.model = None
            self.tokenizer = None

    def generate_response(self, prompt: str, max_length: int = 200) -> str:
        """Generate AI response"""
        if not self.model or not self.tokenizer:
            return self._fallback_response(prompt)

        try:
            # Encode the prompt
            inputs = self.tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=512)

            # Generate response
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                outputs = self.model.generate(
                    inputs,
                    max_length=inputs.shape[1] + max_length,
                    num_return_sequences=1,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.pad_token_id
                )

            # Decode response
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract the new part (response after the prompt)
            if len(response) > len(prompt):
                return response[len(prompt):].strip()
            else:
                return self._fallback_response(prompt)

        except Exception as e:
            print(f"Error generating AI response: {e}")
            return self._fallback_response(prompt)

    def _fallback_response(self, prompt: str) -> str:
        """Rule-based fallback when AI model fails"""
        if 'interaction' in prompt.lower():
            return "Drug interactions can be serious. Please check with a pharmacist or doctor for specific drug combinations."
        elif 'dosage' in prompt.lower():
            return "Dosages should always be determined by a healthcare provider based on individual patient factors."
        elif 'alternative' in prompt.lower():
            return "Alternative medications should be discussed with your doctor or pharmacist to ensure they're appropriate for your condition."
        else:
            return "For specific medical advice, please consult with a healthcare professional."

class DrugChatbotInterface:
    """Simple drug interaction chatbot interface"""

    def __init__(self):
        self.drug_db = DrugDatabase()
        self.ai_model = LocalAIModel()
        self.chat_history = []

    def process_drug_query(self, message: str, history: List) -> Tuple[str, List]:
        """Process drug-related queries"""
        # Add user message to history
        history.append([message, ""])

        try:
            # Determine query type and process accordingly
            if any(word in message.lower() for word in ['interaction', 'interact', 'combine', 'together']):
                response = self._handle_interaction_query(message)
            elif any(word in message.lower() for word in ['dosage', 'dose', 'how much', 'amount']):
                response = self._handle_dosage_query(message)
            elif any(word in message.lower() for word in ['alternative', 'substitute', 'replace', 'instead']):
                response = self._handle_alternative_query(message)
            elif any(word in message.lower() for word in ['extract', 'analyze', 'find drugs']):
                response = self._handle_text_extraction(message)
            else:
                response = self._handle_general_query(message)

            # Update history with response
            history[-1][1] = response

        except Exception as e:
            history[-1][1] = f"Error processing query: {e}"

        return "", history

    def _handle_interaction_query(self, message: str) -> str:
        """Handle drug interaction queries"""
        # Extract drug names from message
        drugs = re.findall(r'\b(?:aspirin|warfarin|ibuprofen|lisinopril|metformin|simvastatin|acetaminophen)\b', message.lower())

        if len(drugs) < 2:
            return "Please specify at least two drugs to check for interactions. \n\nExample: 'Check interactions between aspirin and warfarin'\n\nSupported drugs: aspirin, warfarin, ibuprofen, lisinopril, metformin, simvastatin, acetaminophen"

        interactions = self.drug_db.check_interactions(drugs[:5])

        response = "## 🔍 Drug Interaction Analysis\n\n"

        if interactions:
            response += "### ⚠️ Interactions Found:\n\n"
            for interaction in interactions:
                severity_emoji = "🔴" if interaction['severity'] == 'high' else "🟡"
                response += f"{severity_emoji} **{interaction['drug1'].title()}** ↔ **{interaction['drug2'].title()}**\n"
                response += f"   - Severity: {interaction['severity'].title()}\n"
                response += f"   - Description: {interaction['description']}\n\n"

            response += "### 📋 Recommendations:\n"
            if any(i['severity'] == 'high' for i in interactions):
                response += "- ⚠️ **HIGH RISK**: Consult your doctor immediately\n"
                response += "- Consider alternative medications\n"
            response += "- Monitor for side effects\n"
            response += "- Inform all healthcare providers about all medications\n"
        else:
            response += "### ✅ No Known Major Interactions Found\n\n"
            response += "However, always inform your healthcare providers about all medications you're taking.\n"

        return response

    def _handle_dosage_query(self, message: str) -> str:
        """Handle dosage recommendation queries"""
        # Extract drug name and age
        drugs = re.findall(r'\b(?:aspirin|warfarin|ibuprofen|lisinopril|metformin|simvastatin|acetaminophen)\b', message.lower())
        age_match = re.search(r'\b(\d+)\s*(?:year|yr|y\.o\.|years old)\b', message)

        if not drugs:
            return "Please specify a drug name for dosage information.\n\nSupported drugs: aspirin, warfarin, ibuprofen, lisinopril, metformin, simvastatin, acetaminophen"

        drug = drugs[0]
        drug_info = self.drug_db.get_drug_info(drug)

        if not drug_info:
            return f"Sorry, I don't have information about {drug}."

        age = int(age_match.group(1)) if age_match else None

        response = f"## 💊 Dosage Information for {drug.title()}\n\n"

        # Check age restrictions
        min_age = drug_info.get('age_restrictions', {}).get('min_age', 0)
        if age and age < min_age:
            response += f"⚠️ **Warning**: {drug.title()} is not recommended for patients under {min_age} years old.\n\n"

        # Dosage information
        response += "### 📏 Typical Dosages:\n"
        for age_group, dosage in drug_info['common_dosages'].items():
            response += f"- **{age_group.title()}**: {dosage}\n"

        # Available forms
        response += f"\n### 💊 Available Forms:\n"
        response += f"{', '.join(drug_info['dosage_forms'])}\n"

        # Contraindications
        if drug_info.get('contraindications'):
            response += f"\n### ⚠️ Contraindications:\n"
            for contra in drug_info['contraindications']:
                response += f"- {contra.title()}\n"

        response += f"\n**⚠️ Important**: Always consult a healthcare provider for personalized dosing recommendations."

        return response

    def _handle_alternative_query(self, message: str) -> str:
        """Handle alternative medication queries"""
        drugs = re.findall(r'\b(?:aspirin|warfarin|ibuprofen|lisinopril|metformin|simvastatin|acetaminophen)\b', message.lower())

        if not drugs:
            return "Please specify a drug name to find alternatives.\n\nSupported drugs: aspirin, warfarin, ibuprofen, lisinopril, metformin, simvastatin, acetaminophen"

        drug = drugs[0]
        alternatives = self.drug_db.get_alternatives(drug)
        drug_info = self.drug_db.get_drug_info(drug)

        if not alternatives:
            return f"No alternative information available for {drug}."

        response = f"## 🔄 Alternative Medications for {drug.title()}\n\n"
        response += f"**Drug Category**: {drug_info.get('category', 'Unknown')}\n\n"

        response += f"### 💊 Possible Alternatives:\n"
        for alt in alternatives:
            response += f"- **{alt.title()}**\n"

        response += f"\n### 📋 Important Notes:\n"
        response += f"- Alternatives may have different dosing schedules\n"
        response += f"- May have different side effect profiles\n"
        response += f"- Some may not be suitable for your specific condition\n"
        response += f"- Insurance coverage may vary\n"

        response += f"\n**⚠️ Critical**: Never switch medications without consulting your healthcare provider first."

        return response

    def _handle_text_extraction(self, message: str) -> str:
        """Handle text extraction queries"""
        # Look for quoted text or medical text patterns
        text_match = re.search(r'"([^"]+)"', message)
        if not text_match:
            # Try to find common drug names in the message itself
            found_drugs = re.findall(r'\b(?:aspirin|warfarin|ibuprofen|lisinopril|metformin|simvastatin|acetaminophen)\b', message.lower())
            if found_drugs:
                text_to_analyze = message
            else:
                return 'Please provide text in quotes for analysis.\n\nExample: \'Extract drugs from "Patient takes aspirin 325mg twice daily"\''
        else:
            text_to_analyze = text_match.group(1)

        # Extract drug information
        found_drugs = re.findall(r'\b(?:aspirin|warfarin|ibuprofen|lisinopril|metformin|simvastatin|acetaminophen)\b', text_to_analyze.lower())

        # Extract dosages
        dosage_patterns = re.findall(r'(\d+(?:\.\d+)?)\s*(?:mg|milligram|gram|g)\b', text_to_analyze.lower())

        # Extract frequencies
        frequency_patterns = re.findall(r'\b(?:once|twice|three times|daily|per day|every \d+ hours|q\d+h|bid|tid|qid)\b', text_to_analyze.lower())

        response = "## 📝 Extracted Drug Information\n\n"

        if found_drugs:
            response += "### 💊 Medications Found:\n"
            for i, drug in enumerate(found_drugs):
                response += f"{i+1}. **{drug.title()}**\n"
                if i < len(dosage_patterns):
                    response += f"   - Dosage: {dosage_patterns[i]}mg\n"
                if i < len(frequency_patterns):
                    response += f"   - Frequency: {frequency_patterns[i]}\n"
                response += "\n"
        else:
            response += "No supported medications found in the text.\n"
            response += "Supported drugs: aspirin, warfarin, ibuprofen, lisinopril, metformin, simvastatin, acetaminophen\n"

        if len(found_drugs) > 1:
            response += "\n### 🔍 Interaction Check:\n"
            interactions = self.drug_db.check_interactions(found_drugs)
            if interactions:
                response += "⚠️ Potential interactions detected! See interaction analysis for details.\n"
            else:
                response += "✅ No major interactions detected.\n"

        return response

    def _handle_general_query(self, message: str) -> str:
        """Handle general queries"""
        # Try to use AI model for general responses
        ai_response = self.ai_model.generate_response(f"Medical question: {message}")

        if len(ai_response) > 10:  # If we got a decent AI response
            return f"## 🤖 AI Response:\n\n{ai_response}\n\n**⚠️ Disclaimer**: This information is for educational purposes only. Always consult healthcare professionals for medical advice."
        else:
            # Fallback to rule-based response
            return """## 🏥 General Medical Information

I can help you with:
- Drug interaction checking
- Dosage information
- Alternative medications
- Analyzing medical text

**Supported medications**: aspirin, warfarin, ibuprofen, lisinopril, metformin, simvastatin, acetaminophen

**Example questions**:
- "Check interactions between aspirin and warfarin"
- "What's the dosage for ibuprofen for a 25 year old?"
- "Alternatives to lisinopril"

⚠️ **Important**: This tool is for educational purposes only. Always consult healthcare professionals for medical advice."""

def create_interface():
    """Create the Gradio interface"""

    interface = DrugChatbotInterface()

    with gr.Blocks(title="🏥 Drug Interaction Detection System", theme=gr.themes.Soft()) as app:
        gr.Markdown("""
        # 🏥 Drug Interaction Detection System
        ### No API Key Required - Local AI Processing

        **Features:**
        - 🔍 Drug Interaction Detection
        - 👶 Age-Specific Dosage Information
        - 💊 Alternative Medication Suggestions
        - 📝 Medical Text Analysis
        - 🤖 Local AI Processing (No Internet Required After Setup)

        **Supported Medications**: aspirin, warfarin, ibuprofen, lisinopril, metformin, simvastatin, acetaminophen
        """)

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 💬 Chat with Drug Assistant")
                chatbot_interface = gr.Chatbot(
                    height=500,
                    label="Drug Analysis Assistant"
                )

                msg_input = gr.Textbox(
                    label="Ask about drugs, interactions, dosages, or alternatives",
                    placeholder="Example: 'Check interactions between aspirin and warfarin'",
                    lines=2
                )

                with gr.Row():
                    send_btn = gr.Button("Send 📤", variant="primary")
                    clear_btn = gr.Button("Clear Chat 🗑️")

        with gr.Row():
            with gr.Column():
                gr.Markdown("""
                ### 📋 Quick Examples:

                **🔍 Drug Interactions:**
                - "Check interactions between aspirin and warfarin"
                - "Can I take ibuprofen with lisinopril?"
                - "What happens if I combine aspirin and ibuprofen?"

                **💊 Dosage Information:**
                - "What's the dosage for aspirin for a 25 year old?"
                - "Ibuprofen dose for children"
                - "How much metformin should an adult take?"

                **🔄 Alternative Medications:**
                - "Alternatives to warfarin"
                - "What can I use instead of ibuprofen?"
                - "Substitute for lisinopril"

                **📝 Text Analysis:**
                - 'Extract drugs from "Patient takes metformin 500mg twice daily"'
                - 'Analyze "Morning: aspirin 81mg, Evening: simvastatin 20mg"'
                """)

            with gr.Column():
                gr.Markdown("""
                ### ⚠️ Important Safety Information:

                **🚨 This tool is for educational purposes only**

                **Always consult healthcare professionals for:**
                - Starting new medications
                - Changing dosages
                - Drug interactions
                - Medical emergencies

                **🔴 Seek immediate medical attention if you experience:**
                - Allergic reactions
                - Severe side effects
                - Unusual symptoms
                - Drug overdose

                **📞 Emergency Numbers:**
                - Emergency: 911 (US)
                - Poison Control: 1-800-222-1222 (US)
                """)

        # Event handlers
        msg_input.submit(
            interface.process_drug_query,
            inputs=[msg_input, chatbot_interface],
            outputs=[msg_input, chatbot_interface]
        )

        send_btn.click(
            interface.process_drug_query,
            inputs=[msg_input, chatbot_interface],
            outputs=[msg_input, chatbot_interface]
        )

        clear_btn.click(
            lambda: ([], ""),
            outputs=[chatbot_interface, msg_input]
        )

    return app

# Main execution
if __name__ == "__main__":
    print("🏥 Starting Drug Interaction Detection System...")
    print("📦 Installing required packages...")

    # Create and launch the interface
    app = create_interface()

    # Launch the interface
    app.launch(
        share=True,
        debug=True,
        server_name="0.0.0.0",
        server_port=7860
    )

🏥 Starting Drug Interaction Detection System...
📦 Installing required packages...
Loading AI model... This may take a few minutes the first time.


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ AI model loaded successfully!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e4ecc9a0f286fad10c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
